In [19]:
from neo4j import GraphDatabase
import pandas as pd
from collections import defaultdict, deque
import json
from typing import Dict, List, Set, Tuple
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class FraudDetectionEngine:
    """
    Advanced fraud detection engine using graph algorithms and pattern matching.
    """
    
    def __init__(self, uri: str, user: str, password: str):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))
        self.results = {
            'pattern1_shell_companies': [],
            'pattern2_circular_trade': [],
            'pattern3_hidden_influence': []
        }
    
    def close(self):
        self.driver.close()
    
    # =========================================================================
    # PATTERN 1: SHELL COMPANY DETECTION
    # =========================================================================
    
    def detect_shell_companies(self, min_chain_length: int = 4, max_invoices: int = 2):
        """
        Detects shell company structures using multi-criteria analysis.
        """
        logger.info("🚨 PATTERN 1: Detecting Shell Company Chains...")
        
        query = """
        MATCH (auditor:Auditor)
        WHERE auditor.risk_level = 'HIGH'
        MATCH (auditor)<-[:AUDITED_BY]-(company:Company)
        MATCH path = (company)-[:SUBSIDIARY_OF*3..10]->(root:Company)
        WHERE ALL(node in nodes(path) WHERE (node)-[:AUDITED_BY]->(auditor))
        WITH auditor, path, nodes(path) as chain_companies
        WHERE size(chain_companies) >= $min_chain_length
        UNWIND chain_companies as comp
        OPTIONAL MATCH (comp)-[:ISSUES_TO]->(inv:Invoice)
        WITH auditor, path, chain_companies, comp, count(inv) as invoice_count
        WITH auditor, path, chain_companies, 
             collect({company: comp.company_id, invoices: invoice_count}) as company_stats
        WHERE ALL(stat in company_stats WHERE stat.invoices <= $max_invoices)
        WITH auditor, chain_companies, company_stats,
             reduce(total = 0, stat in company_stats | total + stat.invoices) as total_invoices,
             size(company_stats) as chain_length
        RETURN auditor.auditor_id as auditor_id,
               auditor.risk_level as risk_level,
               [c in chain_companies | c.company_id] as chain,
               company_stats,
               chain_length,
               total_invoices,
               toFloat(total_invoices) / chain_length as avg_invoices
        ORDER BY chain_length DESC, avg_invoices ASC
        """
        
        with self.driver.session() as session:
            result = session.run(query, {
                'min_chain_length': min_chain_length,
                'max_invoices': max_invoices
            })
            
            patterns = []
            for record in result:
                pattern = {
                    'auditor_id': record['auditor_id'],
                    'risk_level': record['risk_level'],
                    'chain': record['chain'],
                    'chain_length': record['chain_length'],
                    'total_invoices': record['total_invoices'],
                    'avg_invoices': round(record['avg_invoices'], 2),
                    'company_details': record['company_stats'],
                    'risk_score': 0.95
                }
                patterns.append(pattern)
                logger.info(f"  🔴 Found shell chain: {len(record['chain'])} companies, "
                          f"Auditor: {record['auditor_id']}, Avg invoices: {pattern['avg_invoices']}")
            
            self.results['pattern1_shell_companies'] = patterns
            logger.info(f"  ✅ Total shell company patterns detected: {len(patterns)}")
            return patterns
    
    def update_shell_company_risk_scores(self):
        if not self.results['pattern1_shell_companies']:
            logger.warning("  ⚠️  No shell companies to update")
            return
        
        logger.info("  📝 Updating risk scores for shell companies...")
        query = """
        UNWIND $patterns as pattern
        UNWIND pattern.chain as company_id
        MATCH (c:Company {company_id: company_id})
        SET c.risk_score = $risk_score
        RETURN count(c) as updated_count
        """
        
        with self.driver.session() as session:
            result = session.run(query, {
                'patterns': self.results['pattern1_shell_companies'],
                'risk_score': 0.95
            })
            count = result.single()['updated_count']
            logger.info(f"  ✅ Updated risk_score for {count} companies to 0.95")
    
    # =========================================================================
    # PATTERN 2: CIRCULAR TRADE DETECTION
    # =========================================================================
    
    def detect_circular_trade(self, min_cycle_size: int = 3, min_volume: int = 80):
        logger.info("⚠️  PATTERN 2: Detecting Circular Trade Patterns...")
        
        query = """
        MATCH path = (c1:Company)-[:SUPPLIES]->(c2:Company)-[:SUPPLIES]->(c3:Company)
        WHERE c1 <> c2 AND c2 <> c3 AND c1 <> c3
        MATCH (c3)-[r_close:SUPPLIES]->(c1)
        WITH c1, c2, c3, 
            [c1, c2, c3] as cycle_companies,
            relationships(path) + [r_close] as cycle_relationships
        WITH cycle_companies, 
            reduce(vols = [], r in cycle_relationships | vols + [r.annual_volume]) as volumes
        WHERE ALL(v in volumes WHERE v >= $min_volume)
        WITH cycle_companies, volumes,
            reduce(total = 0.0, v in volumes | total + v) as total_volume,
            size(volumes) as cycle_length
        UNWIND cycle_companies as comp
        OPTIONAL MATCH (comp)-[:SUPPLIES]->(external:Company)
        WHERE NOT external IN cycle_companies
        WITH cycle_companies, volumes, total_volume, cycle_length, 
            count(DISTINCT external) as external_connections
        WITH cycle_companies, volumes, total_volume, cycle_length, external_connections,
            toFloat(cycle_length) / (cycle_length + external_connections + 1) as isolation_score
        WHERE isolation_score > 0.5
        RETURN [c in cycle_companies | c.company_id] as cycle,
            cycle_length,
            volumes,
            total_volume,
            toFloat(total_volume) / cycle_length as avg_volume,
            external_connections,
            isolation_score
        ORDER BY isolation_score DESC, avg_volume DESC
        LIMIT 100
        """
        
        with self.driver.session() as session:
            result = session.run(query, {'min_volume': min_volume})
            patterns = []
            seen_cycles = set()
            
            for record in result:
                cycle = record['cycle']
                cycle_sig = tuple(sorted(cycle))
                if cycle_sig in seen_cycles:
                    continue
                seen_cycles.add(cycle_sig)
                
                pattern = {
                    'cycle': cycle,
                    'cycle_length': record['cycle_length'],
                    'volumes': record['volumes'],
                    'total_volume': record['total_volume'],
                    'avg_volume': round(record['avg_volume'], 2),
                    'external_connections': record['external_connections'],
                    'isolation_score': round(record['isolation_score'], 3),
                    'risk_score': 0.80 + (0.15 * record['isolation_score'])
                }
                patterns.append(pattern)
                logger.info(f"  🟡 Found circular trade: {len(cycle)} companies, "
                          f"Isolation: {pattern['isolation_score']:.2f}, "
                          f"Avg volume: {pattern['avg_volume']}")
            
            self.results['pattern2_circular_trade'] = patterns
            logger.info(f"  ✅ Total circular trade patterns detected: {len(patterns)}")
            return patterns
    
    def detect_circular_trade_advanced(self, min_cycle_size: int = 3):
        logger.info("  🔍 Running advanced cycle detection (4-6 companies)...")
        
        query = """
        MATCH path = (c1:Company)-[:SUPPLIES]->(c2:Company)-[:SUPPLIES]->
                    (c3:Company)-[:SUPPLIES]->(c4:Company)-[:SUPPLIES]->(c1)
        WHERE c1 <> c2 AND c2 <> c3 AND c3 <> c4 AND c4 <> c1
        AND c1 <> c3 AND c2 <> c4
        WITH [c1, c2, c3, c4] as cycle_companies,
            relationships(path) as path_rels
        WITH cycle_companies, 
            reduce(vols = [], r in path_rels | vols + [r.annual_volume]) as volumes
        WHERE ALL(v in volumes WHERE v >= 80)
        WITH [c in cycle_companies | c.company_id] as cycle,
            size(cycle_companies) as cycle_length,
            volumes,
            reduce(total = 0.0, v in volumes | total + v) as total_volume
        RETURN cycle, cycle_length, volumes, total_volume
        LIMIT 50
        UNION
        MATCH path = (c1:Company)-[:SUPPLIES]->(c2:Company)-[:SUPPLIES]->
                    (c3:Company)-[:SUPPLIES]->(c4:Company)-[:SUPPLIES]->
                    (c5:Company)-[:SUPPLIES]->(c1)
        WHERE c1 <> c2 AND c2 <> c3 AND c3 <> c4 AND c4 <> c5 AND c5 <> c1
        AND c1 <> c3 AND c1 <> c4 AND c2 <> c4 AND c2 <> c5 AND c3 <> c5
        WITH [c1, c2, c3, c4, c5] as cycle_companies,
            relationships(path) as path_rels
        WITH cycle_companies, 
            reduce(vols = [], r in path_rels | vols + [r.annual_volume]) as volumes
        WHERE ALL(v in volumes WHERE v >= 80)
        WITH [c in cycle_companies | c.company_id] as cycle,
            size(cycle_companies) as cycle_length,
            volumes,
            reduce(total = 0.0, v in volumes | total + v) as total_volume
        RETURN cycle, cycle_length, volumes, total_volume
        LIMIT 30
        """
        
        with self.driver.session() as session:
            result = session.run(query)
            patterns = []
            seen_cycles = set()
            
            for record in result:
                cycle = record['cycle']
                cycle_sig = tuple(sorted(cycle))
                if cycle_sig not in seen_cycles:
                    seen_cycles.add(cycle_sig)
                    pattern = {
                        'cycle': cycle,
                        'cycle_length': record['cycle_length'],
                        'volumes': record['volumes'],
                        'total_volume': record['total_volume'],
                        'avg_volume': round(record['total_volume'] / record['cycle_length'], 2),
                        'risk_score': 0.85
                    }
                    patterns.append(pattern)
            
            self.results['pattern2_circular_trade'].extend(patterns)
            logger.info(f"  ✅ Found {len(patterns)} additional cycles (4-5 companies)")
    
    def update_circular_trade_risk_scores(self):
        if not self.results['pattern2_circular_trade']:
            logger.warning("  ⚠️  No circular trade patterns to update")
            return
        
        logger.info("  📝 Updating risk scores for circular trade participants...")
        query = """
        UNWIND $patterns as pattern
        UNWIND pattern.cycle as company_id
        MATCH (c:Company {company_id: company_id})
        SET c.risk_score = CASE 
            WHEN c.risk_score IS NULL OR c.risk_score < pattern.risk_score 
            THEN pattern.risk_score
            ELSE c.risk_score
        END
        RETURN count(c) as updated_count
        """
        
        with self.driver.session() as session:
            result = session.run(query, {'patterns': self.results['pattern2_circular_trade']})
            count = result.single()['updated_count']
            logger.info(f"  ✅ Updated risk_score for {count} companies to 0.80-0.90")
    
    # =========================================================================
    # PATTERN 3: HIDDEN INFLUENCE DETECTION (GDS REQUIRED)
    # =========================================================================
    
    def calculate_shareholder_influence_pagerank(self):
        logger.info("  📊 Calculating PageRank for shareholder influence...")
        
        with self.driver.session() as session:
            # ✅ FIXED: No nodeProperties (causes String loading error)
            create_graph = """
            CALL gds.graph.project(
                'ownership-graph',
                ['Shareholder', 'Company'],
                {
                    OWNS_SHARE: {
                        properties: 'percentage',
                        orientation: 'NATURAL'
                    }
                }
            )
            YIELD graphName, nodeCount, relationshipCount
            RETURN graphName, nodeCount, relationshipCount
            """
            
            graph_info = session.run(create_graph).single()
            logger.info(f"  ✅ Graph created: {graph_info['graphName']}, "
                    f"Nodes: {graph_info['nodeCount']}, "
                    f"Relationships: {graph_info['relationshipCount']}")
            
            pagerank_query = """
            CALL gds.pageRank.stream('ownership-graph', {
                maxIterations: 20,
                dampingFactor: 0.85,
                relationshipWeightProperty: 'percentage'
            })
            YIELD nodeId, score
            WITH gds.util.asNode(nodeId) as node, score
            WHERE node:Shareholder AND score > 0.01
            RETURN node.shareholder_id as shareholder_id, score as pagerank_score
            ORDER BY score DESC
            """
            
            result = session.run(pagerank_query)
            influence_scores = {}
            for record in result:
                influence_scores[record['shareholder_id']] = record['pagerank_score']
            
            top_influencers = sorted(influence_scores.items(), key=lambda x: x[1], reverse=True)[:3]
            for sid, score in top_influencers:
                logger.info(f"  🥇 Top influencer: {sid} (PageRank: {score:.4f})")
            
            logger.info(f"  ✅ PageRank calculated for {len(influence_scores)} shareholders")
            session.run("CALL gds.graph.drop('ownership-graph') YIELD graphName")
            logger.info("  🧹 GDS graph dropped")
            
            return influence_scores

    def detect_hidden_influence(self, min_ownership: float = 25.0, min_concentration: float = 80.0):
        """
        Detects hidden influence patterns through concentrated supply chains.
        """
        logger.info("✨ PATTERN 3: Detecting Hidden Influence Patterns...")
        
        # Calculate PageRank influence scores
        influence_scores = self.calculate_shareholder_influence_pagerank()
        
        query = """
        MATCH (shareholder:Shareholder)-[owns:OWNS_SHARE]->(supplier:Company)
        WHERE owns.percentage > $min_ownership
        MATCH (supplier)-[supplies:SUPPLIES]->(target:Company)
        MATCH (supplier)-[:ISSUES_TO]->(inv1:Invoice)<-[:PAYS]-(target)
        WITH shareholder, supplier, target, owns.percentage as ownership_pct,
            count(DISTINCT inv1) as supplier_invoices
        MATCH (target)-[:PAYS]->(inv2:Invoice)
        WITH shareholder, supplier, target, ownership_pct, supplier_invoices,
            count(DISTINCT inv2) as total_invoices
        WITH shareholder, supplier, target, ownership_pct, supplier_invoices, total_invoices,
            toFloat(supplier_invoices) / total_invoices * 100 as concentration_pct
        WHERE concentration_pct >= $min_concentration
        
        // CRITICAL FIX: Check for NO direct supply relationship
        OPTIONAL MATCH (shareholder)-[direct:SUPPLIES]->(target)
        WITH shareholder, supplier, target, ownership_pct, supplier_invoices, 
            total_invoices, concentration_pct, 
            coalesce(count(direct), 0) as direct_supply_count
        WHERE direct_supply_count = 0  // Only hidden influence paths
        
        RETURN shareholder.shareholder_id as shareholder_id,
            shareholder.name as shareholder_name,
            shareholder.type as shareholder_type,
            supplier.company_id as supplier_id,
            supplier.name as supplier_name,
            target.company_id as target_id,
            target.name as target_name,
            ownership_pct,
            supplier_invoices,
            total_invoices,
            concentration_pct
        ORDER BY concentration_pct DESC, ownership_pct DESC
        LIMIT 50
        """
        
        with self.driver.session() as session:
            result = session.run(query, {
                'min_ownership': min_ownership,
                'min_concentration': min_concentration
            })
            
            patterns = []
            raw_matches = result.peek() is not None  # Debug: check if any results
            
            for record in result:
                shareholder_id = record['shareholder_id']
                influence_score = influence_scores.get(shareholder_id, 0.1)
                
                ownership_factor = min(record['ownership_pct'] / 50.0, 1.0)
                concentration_factor = record['concentration_pct'] / 100.0
                opportunity_score = 0.7 + (0.3 * (
                    0.4 * influence_score +
                    0.3 * ownership_factor +
                    0.3 * concentration_factor
                ))
                
                pattern = {
                    'shareholder_id': shareholder_id,
                    'shareholder_name': record['shareholder_name'],
                    'shareholder_type': record['shareholder_type'],
                    'supplier_id': record['supplier_id'],
                    'supplier_name': record['supplier_name'],
                    'target_id': record['target_id'],
                    'target_name': record['target_name'],
                    'ownership_pct': round(record['ownership_pct'], 2),
                    'supplier_invoices': record['supplier_invoices'],
                    'total_invoices': record['total_invoices'],
                    'concentration_pct': round(record['concentration_pct'], 2),
                    'influence_score': round(influence_score, 3),
                    'opportunity_score': round(opportunity_score, 3)
                }
                patterns.append(pattern)
                
                logger.info(f"  🟢 Hidden influence: {shareholder_id} owns {pattern['ownership_pct']}% "
                        f"of {record['supplier_id']}, supplies {pattern['concentration_pct']}% "
                        f"to {record['target_id']} (score: {pattern['opportunity_score']:.3f})")
            
            logger.info(f"  📊 Raw matches found: {raw_matches}, Final patterns: {len(patterns)}")
            self.results['pattern3_hidden_influence'] = patterns
            logger.info(f"  ✅ Total hidden influence patterns detected: {len(patterns)}")
            return patterns


    
#     def detect_hidden_influence(self, min_ownership: float = 25.0, min_concentration: float = 80.0):
#         """
#         Detects hidden influence patterns through concentrated supply chains.
#         """
#         logger.info("✨ PATTERN 3: Detecting Hidden Influence Patterns...")
        
#         # Calculate PageRank influence scores
#         influence_scores = self.calculate_shareholder_influence_pagerank()
        
#         query = """
#         MATCH (shareholder:Shareholder)-[owns:OWNS_SHARE]->(supplier:Company)
#         WHERE owns.percentage > $min_ownership
#         MATCH (supplier)-[supplies:SUPPLIES]->(target:Company)
#         MATCH (supplier)-[:ISSUES_TO]->(inv1:Invoice)<-[:PAYS]-(target)
#         WITH shareholder, supplier, target, owns.percentage as ownership_pct,
#             count(DISTINCT inv1) as supplier_invoices
#         MATCH (target)-[:PAYS]->(inv2:Invoice)
#         WITH shareholder, supplier, target, ownership_pct, supplier_invoices,
#             count(DISTINCT inv2) as total_invoices
#         WITH shareholder, supplier, target, ownership_pct, supplier_invoices, total_invoices,
#             toFloat(supplier_invoices) / total_invoices * 100 as concentration_pct
#         WHERE concentration_pct >= $min_concentration
#         OPTIONAL MATCH (shareholder)-[:SUPPLIES]->(target)
#         WITH shareholder, supplier, target, ownership_pct, supplier_invoices, 
#             total_invoices, concentration_pct, count(shareholder) as direct_supply_count
#         WITH shareholder, supplier, target, ownershippct, supplierinvoices, totalinvoices, concentrationpct, 
#             coalesce(count(direct), 0) as directsupplycount
#         WHERE direct_supply_count = 0
#         RETURN shareholder.shareholder_id as shareholder_id,
#             shareholder.name as shareholder_name,
#             shareholder.type as shareholder_type,
#             supplier.company_id as supplier_id,
#             supplier.name as supplier_name,
#             target.company_id as target_id,
#             target.name as target_name,
#             ownership_pct,
#             supplier_invoices,
#             total_invoices,
#             concentration_pct
#         ORDER BY concentration_pct DESC, ownership_pct DESC
#         LIMIT 50
#         """
        

#         with self.driver.session() as session:
#             result = session.run(query, {
#                 'min_ownership': min_ownership,
#                 'min_concentration': min_concentration
#             })
            
#             patterns = []
#             for record in result:
#                 shareholder_id = record['shareholder_id']
#                 influence_score = influence_scores.get(shareholder_id, 0.1)
                
#                 ownership_factor = min(record['ownership_pct'] / 50.0, 1.0)
#                 concentration_factor = record['concentration_pct'] / 100.0
#                 opportunity_score = 0.7 + (0.3 * (
#                     0.4 * influence_score +
#                     0.3 * ownership_factor +
#                     0.3 * concentration_factor
#                 ))
                
#                 pattern = {
#                     'shareholder_id': shareholder_id,
#                     'shareholder_name': record['shareholder_name'],
#                     'shareholder_type': record['shareholder_type'],
#                     'supplier_id': record['supplier_id'],
#                     'supplier_name': record['supplier_name'],
#                     'target_id': record['target_id'],
#                     'target_name': record['target_name'],
#                     'ownership_pct': round(record['ownership_pct'], 2),
#                     'supplier_invoices': record['supplier_invoices'],
#                     'total_invoices': record['total_invoices'],
#                     'concentration_pct': round(record['concentration_pct'], 2),
#                     'influence_score': round(influence_score, 3),
#                     'opportunity_score': round(opportunity_score, 3)
#                 }
#                 patterns.append(pattern)
                
#                 logger.info(f"  🟢 Hidden influence: {shareholder_id} owns {pattern['ownership_pct']}% "
#                           f"of {record['supplier_id']}, supplies {pattern['concentration_pct']}% "
#                           f"to {record['target_id']} (score: {pattern['opportunity_score']:.3f})")
            
#             self.results['pattern3_hidden_influence'] = patterns
#             logger.info(f"  ✅ Total hidden influence patterns detected: {len(patterns)}")
#             return patterns
    
    def update_hidden_influence_opportunity_scores(self):
        if not self.results['pattern3_hidden_influence']:
            logger.warning("  ⚠️  No hidden influence patterns to update")
            return
        
        logger.info("  📝 Updating opportunity scores for hidden influence...")
        
        query_shareholders = """
        UNWIND $patterns as pattern
        MATCH (s:Shareholder {shareholder_id: pattern.shareholder_id})
        SET s.opportunity_score = CASE
            WHEN s.opportunity_score IS NULL OR s.opportunity_score < pattern.opportunity_score
            THEN pattern.opportunity_score
            ELSE s.opportunity_score
        END
        RETURN count(s) as updated_count
        """
        
        query_targets = """
        UNWIND $patterns as pattern
        MATCH (c:Company {company_id: pattern.target_id})
        SET c.opportunity_score = CASE
            WHEN c.opportunity_score IS NULL OR c.opportunity_score < pattern.opportunity_score
            THEN pattern.opportunity_score
            ELSE c.opportunity_score
        END
        RETURN count(c) as updated_count
        """
        
        with self.driver.session() as session:
            result1 = session.run(query_shareholders, {'patterns': self.results['pattern3_hidden_influence']})
            count1 = result1.single()['updated_count']
            
            result2 = session.run(query_targets, {'patterns': self.results['pattern3_hidden_influence']})
            count2 = result2.single()['updated_count']
            
            logger.info(f"  ✅ Updated {count1} shareholders and {count2} companies to >0.70 opportunity score")
    
    # =========================================================================
    # EXECUTION & REPORTING
    # =========================================================================
    
    def run_all_detections(self):
        logger.info("\n" + "=" * 80)
        logger.info("🚀 STARTING COMPREHENSIVE FRAUD DETECTION")
        logger.info("=" * 80 + "\n")
        
        # Pattern 1: Shell Companies
        self.detect_shell_companies(min_chain_length=4, max_invoices=2)
        self.update_shell_company_risk_scores()
        
        # Pattern 2: Circular Trade
        self.detect_circular_trade(min_cycle_size=3, min_volume=80)
        self.detect_circular_trade_advanced()
        self.update_circular_trade_risk_scores()
        
        # Pattern 3: Hidden Influence (GDS REQUIRED)
        self.detect_hidden_influence(min_ownership=25.0, min_concentration=80.0)
        self.update_hidden_influence_opportunity_scores()
        
        logger.info("\n" + "=" * 80)
        logger.info("✅ FRAUD DETECTION COMPLETE - ALL PATTERNS PROCESSED")
        logger.info("=" * 80)
        
        return self.results
    
    def generate_report(self, output_file: str = 'fraud_detection_report.json'):
        logger.info(f"\n📄 Generating report: {output_file}")
        
        report = {
            'summary': {
                'shell_companies': len(self.results['pattern1_shell_companies']),
                'circular_trade_patterns': len(self.results['pattern2_circular_trade']),
                'hidden_influence_patterns': len(self.results['pattern3_hidden_influence'])
            },
            'patterns': self.results,
            'timestamp': pd.Timestamp.now().isoformat()
        }
        
        with open(output_file, 'w') as f:
            json.dump(report, f, indent=2)
        
        logger.info(f"  ✅ Report saved to {output_file}")
        
        print("\n" + "=" * 80)
        print("📊 FRAUD DETECTION SUMMARY")
        print("=" * 80)
        print(f"🚨 Shell Company Chains: {report['summary']['shell_companies']}")
        print(f"⚠️  Circular Trade Patterns: {report['summary']['circular_trade_patterns']}")
        print(f"✨ Hidden Influence Patterns: {report['summary']['hidden_influence_patterns']}")
        print("=" * 80 + "\n")
        return report

def main():
    NEO4J_URI = "bolt://localhost:7687"
    NEO4J_USER = "neo4j"
    NEO4J_PASSWORD = "password"  
    
    engine = FraudDetectionEngine(NEO4J_URI, NEO4J_USER, NEO4J_PASSWORD)
    
    try:
        results = engine.run_all_detections()
        engine.generate_report('fraud_detection_report.json')
    finally:
        engine.close()

if __name__ == "__main__":
    main()


INFO:__main__:
INFO:__main__:🚀 STARTING COMPREHENSIVE FRAUD DETECTION
INFO:__main__:================================================================================

INFO:__main__:🚨 PATTERN 1: Detecting Shell Company Chains...
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A0, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A0, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A0, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A1, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A1, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A1, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A1, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A1, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 companies, Auditor: A1, Avg invoices: 1.0
INFO:__main__:  🔴 Found shell chain: 5 compa


📊 FRAUD DETECTION SUMMARY
🚨 Shell Company Chains: 42
⚠️  Circular Trade Patterns: 25
✨ Hidden Influence Patterns: 25

